# Demo: Logging System and Analysis Object Representation

This notebook demonstrates the new logging system and `__repr__`/`__str__` implementations for analysis objects.

## Features Demonstrated

1. **ConsoleLogger**: Consistent, beautiful logging with symbols
2. **Analysis `__repr__`**: Constructor-style representation
3. **Analysis `__str__`**: Human-readable configuration display

These improvements make the package more user-friendly and easier to debug.

In [ ]:
# Import required modules
import numpy as np
from pathlib import Path

from ldk.utils.logging import ConsoleLogger, log_section, log_info, log_success, log_warning, log_error, log_progress
from ldk.analysis import FunctionalNetworkMapping, StructuralNetworkMapping, RegionalDamage, AtlasAggregation

## Part 1: Console Logger Demonstration

The `ConsoleLogger` provides consistent formatting with emojis and proper indentation.

In [ ]:
# Create a logger instance
logger = ConsoleLogger(verbose=True, width=70)

# Section headers
logger.section("PROCESSING PIPELINE")

# Info messages
logger.info("Starting data processing...")
logger.info("Loading connectome data", indent_level=1)
logger.info("Validating input files", indent_level=1)

In [ ]:
# Success messages with details
logger.success("Data loaded successfully", details={
    "n_subjects": 1000,
    "n_voxels": 228483,
    "memory_usage_mb": 15.3,
    "load_time_sec": 2.45
})

In [ ]:
# Subsection headers
logger.subsection("Running Analysis")

# Progress messages
logger.progress("Processing batch", current=1, total=5)
logger.progress("Processing batch", current=2, total=5)
logger.progress("Processing batch", current=3, total=5)

# Or with percentage
logger.progress("Computing correlations", percent=75.5)

In [ ]:
# Warning messages
logger.warning("Lesion size smaller than 100 voxels")
logger.warning("Some voxels fall outside brain mask", indent_level=1)

In [ ]:
# Result summaries
logger.result_summary("Analysis Results", {
    "Mean correlation": 0.4523,
    "Std correlation": 0.1234,
    "Max correlation": 0.8912,
    "Min correlation": -0.3456,
    "N significant voxels": 12458
})

### Using Convenience Functions

For quick logging, use the convenience functions:

In [ ]:
log_section("BATCH PROCESSING")
log_info("Processing 10 subjects in parallel")
log_progress("Subjects processed", current=7, total=10)
log_success("All subjects completed successfully!")
log_warning("2 subjects had small lesions (< 50 voxels)")

## Part 2: Analysis Object Representation

All analysis classes now have informative `__repr__` and `__str__` methods.

### FunctionalNetworkMapping

In [ ]:
# Create analysis object
flnm = FunctionalNetworkMapping(
    connectome_path="/path/to/connectome_batches/",
    method="boes",
    compute_t_map=True,
    t_threshold=2.0,
    verbose=True
)

# repr() - constructor-style representation
print("repr() output:")
print(repr(flnm))
print()

# str() / print() - human-readable display
print("print() output:")
print(flnm)

### StructuralNetworkMapping

In [ ]:
# Create analysis object
slnm = StructuralNetworkMapping(
    tractogram_path="/path/to/whole_brain.tck",
    whole_brain_tdi="/path/to/tdi_map.nii.gz",
    atlas_path="schaefer100",
    compute_lesioned=True,
    n_jobs=4,
    check_dependencies=False
)

print("repr() output:")
print(repr(slnm))
print()

print("print() output:")
print(slnm)

### RegionalDamage

In [ ]:
# Create analysis object
rd = RegionalDamage(
    threshold=0.5,
    atlas_names=["AAL3", "Schaefer2018_100"]
)

print("repr() output:")
print(repr(rd))
print()

print("print() output:")
print(rd)

### AtlasAggregation

In [ ]:
# Create analysis object
aa = AtlasAggregation(
    source="lesion_img",
    aggregation="mean",
    threshold=0.3,
    atlas_names=["AAL3", "Schaefer2018_200"]
)

print("repr() output:")
print(repr(aa))
print()

print("print() output:")
print(aa)

## Part 3: Practical Use Cases

These features make debugging and logging much easier:

### Use Case 1: Debugging Analysis Configuration

In [ ]:
# When debugging, quickly check your analysis configuration
analyses = [
    FunctionalNetworkMapping("/data/connectome.h5", method="boes"),
    FunctionalNetworkMapping("/data/connectome.h5", method="pini", pini_percentile=30),
    RegionalDamage(threshold=0.5)
]

print("Configured analyses:")
for i, analysis in enumerate(analyses, 1):
    print(f"\n{i}. {repr(analysis)}")

### Use Case 2: Workflow with Logging

In [ ]:
logger = ConsoleLogger(verbose=True, width=70)

logger.section("LESION ANALYSIS WORKFLOW")

# Step 1: Setup
logger.subsection("Step 1: Configuration")
analysis = FunctionalNetworkMapping(
    connectome_path="/data/connectome.h5",
    method="boes",
    compute_t_map=True,
    t_threshold=2.0
)
logger.info(f"Analysis configured: {analysis.__class__.__name__}")
logger.info("Parameters:", indent_level=1)
for key, value in analysis._get_parameters().items():
    logger.info(f"{key} = {value}", indent_level=2)

# Step 2: Validation
logger.subsection("Step 2: Validation")
logger.info("Checking connectome file...")
logger.info("Checking lesion registration...")
logger.success("All validations passed")

# Step 3: Processing
logger.subsection("Step 3: Processing")
logger.info("Extracting lesion timeseries...")
logger.info("Computing correlations...", indent_level=1)
logger.progress("Batch processing", current=1, total=5)
logger.progress("Batch processing", current=5, total=5)
logger.success("Processing complete", details={
    "processing_time": 42.5,
    "n_correlations": 228483,
    "memory_peak_mb": 15.2
})

logger.section("WORKFLOW COMPLETE ✓")

### Use Case 3: Parameter Comparison

In [ ]:
# Compare different analysis configurations
configs = [
    FunctionalNetworkMapping("/data/connectome.h5", method="boes"),
    FunctionalNetworkMapping("/data/connectome.h5", method="pini", pini_percentile=20),
    FunctionalNetworkMapping("/data/connectome.h5", method="pini", pini_percentile=50),
]

logger.section("PARAMETER COMPARISON")
for i, config in enumerate(configs, 1):
    logger.subsection(f"Configuration {i}")
    print(config)
    logger.blank_line()

## Summary

### Key Benefits

1. **Consistent User Experience**: All messages use the same formatting and symbols
2. **Easy Debugging**: `repr()` shows exactly how to recreate an analysis object
3. **Readable Output**: `print()` gives human-friendly configuration display
4. **Progress Tracking**: Clear progress indicators with ETAs and percentages
5. **Professional Look**: Emoji symbols make output visually appealing and scannable

### Next Steps

- The logging system will be integrated into all analysis modules
- `verbose` flags will control output verbosity
- Analysis provenance will automatically capture parameters via `_get_parameters()`